In [ ]:
# Figuring out the cSPARC .cs file format

import numpy as np
import matplotlib.pyplot as plt

# For a non-memory-mapped array
data = np.fromfile('./csparc/J6_particles.cs', dtype=float)  # Specify the data type explicitly

# Reshape the data to match the type_info structure
print(data)
print(f"Data type: {data.dtype}")
print(f"Shape: {data.shape}")




In [ ]:
import numpy as np
import re

try:
    # Load the data
    data = np.load('./csparc/J6_particles.cs')
    
    if isinstance(data, np.ndarray) and data.dtype.names is not None:
        print("Column Headers:")
        print(data.dtype.names)
    else:
        print("The file does not contain structured array headers.")
except FileNotFoundError:
    print("File not found. Please check the file path.")
except Exception as e:
    print(f"An error occurred: {e}")

# Print the first few rows
print("First few rows of data:")
print(data[:5])
print("Check to see if it is the same particle order as the cryoDRGN")
print(data[65000:65010])

print(f"Data type: {data.dtype}")
print(f"Shape: {data.shape}")

# Extract component values from each mode
components_mode0 = data['components_mode_0/value']
components_mode1 = data['components_mode_1/value']
components_mode2 = data['components_mode_2/value']

# Stack the three components along a new axis to form a 3D vector for each row
components_vectors = np.stack([components_mode0, components_mode1, components_mode2], axis=1)

# test to see if it works
print("Shape of components_vectors:", components_vectors.shape)
print("\nFirst latent space vectors:")
print(components_vectors[:5])










In [ ]:
import numpy as np
import re

def extract_latent_space_vectors(structured_array):
    """
    Extracts component values from different modes and combines them into vectors.
    
    Parameters:
        - structured_array: A structured NumPy array containing component data.
        
    Returns:
        An n-dimensional numpy ndarray where each row is a vector of components from each mode.
    """
    # Identify all columns matching the pattern 'component_mode_X/value'
    component_columns = [col for col in structured_array.dtype.names if re.match(r'components_mode_\d+/value', col)]
    
    if not component_columns:
        raise ValueError("No component columns found in the structured array.")
    
    # Extract values from each identified column
    components = []
    for col in component_columns:
        # Extract the mode number from the column name
        mode_number = int(re.search(r'\d+', col).group())
        # Append the component values to the list
        components.append(structured_array[col])
    
    # Stack the component arrays along a new axis to form vectors
    stacked_components = np.stack(components, axis=1)
    
    return stacked_components

# Example usage:
# Load your structured array (replace 'your_data.cs' with your actual file path)


# Extract components into vectors
z = extract_latent_space_vectors(data) 
print("Shape of the resulting array:", z.shape)
print("First few vectors:\n", z[:5, :])



In [ ]:
# Looking firstly at latent space vector magnitudes.

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# this function is the same as np.linalg.norm
def calculate_vector_magnitudes(vectors):
    """
    Calculates the magnitudes of vectors extracted from a NumPy array.
    
    Parameters:
        - vectors: A n-D numpy ndarray where each row is a vector.
        
    Returns:
        A 1D numpy ndarray where each element is the magnitude of the corresponding vector.
    """
    
    # Calculate the magnitude of each vector
    squared = np.square(vectors)
    sum_squared = np.sum(squared, axis=1)
    magnitudes = np.sqrt(sum_squared)
    
    return magnitudes

# Example usage:

mags = calculate_vector_magnitudes(z)
print("Magnitudes:", mags[:5])

mags2 = np.linalg.norm(z, axis=1)
print("Magnitudes using np.linalg.norm:", mags2[:5])



# Separate the first 65,000 entries
OA = z[:65000]
OEA = z[65000:]

# Calculate the magnitude of each vector in your dataset
OA_magnitudes = calculate_vector_magnitudes(OA)
OEA_magnitudes = calculate_vector_magnitudes(OEA)



# Plot histograms for both subsets
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
plt.hist(OA_magnitudes, bins=500, alpha=0.5, label='OA')
plt.hist(OEA_magnitudes, bins=500, alpha=0.5, label='OEA')
#plt.hist(OEA_magnitudes, bins=500,  alpha=0.5,label='OEA', color='orange')   
plt.title('Histogram of Vector Magnitudes')
plt.xlabel('Magnitude')
plt.xlim(0,200)
#plt.ylim(0, 6000)
plt.ylabel('Frequency')
plt.legend()

# Boxplot for both subsets
plt.subplot(1,2,2)
plt.violinplot([OA_magnitudes, OEA_magnitudes], showmeans=True)
plt.title('ViolinPlot of Vector Magnitudes')
plt.xticks([1, 2], ['OA', 'OEA'])

plt.tight_layout()

plt.show()

# Perform t-test to compare means of both subsets
t_stat, p_value = stats.ttest_ind(OA_magnitudes, OEA_magnitudes)
print('T-test')
print('T-Statistic:', t_stat)
print('P-Value:', p_value)

u_stat, p_value = stats.mannwhitneyu(OA_magnitudes, OEA_magnitudes, alternative='two-sided')
print('Mann-Whitney U-Test')
print('U-Statistic:', u_stat)
print('P-Value:', p_value)

f_stat, p_value = stats.f_oneway(OA_magnitudes, OEA_magnitudes)
print('ANOVA Test')
print('F-Statistic:', f_stat)
print('P-Value:', p_value)

mean_OA = np.mean(OA_magnitudes)
mean_OEA = np.mean(OEA_magnitudes)
print('Mean OA Magnitude:', mean_OA, 'SD', np.std(OA_magnitudes))
print('Mean OEA Magnitude:', mean_OEA, 'SD', np.std(OEA_magnitudes))







In [ ]:
import umap
from pathlib import Path
import joblib

# Create a function to perform UMAP and save the results in a .pkl file
def perform_and_save_umap(data, filename):
    if filename.exists():
        print("Loading UMAP embeddings from disk...")
        data_umap = joblib.load(filename)
    else:
        print("Performing UMAP and saving the results to disk...")
        umap_reducer = umap.UMAP(n_components=2, metric='euclidean', min_dist=0.1, n_neighbors=50, spread=1.5, learning_rate=0.5, negative_sample_rate=10, init='pca', random_state=42)
        data_umap = umap_reducer.fit_transform(data)
        joblib.dump(data_umap, filename)
    return data_umap

# Define filenames for the UMAP embeddings
#filename_OA_subset_A = Path("umap_embeddings_OA_subset_A.pkl")
#filename_OA_subset_B = Path("umap_embeddings_OA_subset_B.pkl")
#filename_OEA_subset_A = Path("umap_embeddings_OEA_subset_A.pkl")
#filename_OEA_subset_B = Path("umap_embeddings_OEA_subset_B.pkl")
filename_OA = Path("umap_embeddings_OA.pkl")
filename_OEA = Path("umap_embeddings_OEA.pkl")

# Perform UMAP on subsets and save the results to disk
#data_umap_OA_subset_A = perform_and_save_umap(OA_subset_A, filename_OA_subset_A)
#data_umap_OA_subset_B = perform_and_save_umap(OA_subset_B, filename_OA_subset_B)
#data_umap_OEA_subset_A = perform_and_save_umap(OEA_subset_A, filename_OEA_subset_A)
#data_umap_OEA_subset_B = perform_and_save_umap(OEA_subset_B, filename_OEA_subset_B)
data_umap_OA = perform_and_save_umap(OA, filename_OA)
data_umap_OEA = perform_and_save_umap(OEA, filename_OEA)


In [ ]:
import tensorflow as tf
from sklearn.manifold import TSNE
import joblib
from pathlib import Path

# Create a function to perform tSNE and save the results in a .pkl file
def perform_and_save_tsne(data, filename, perplexity=100, n_iter=1000):
    if filename.exists():
        print("Loading t-SNE embeddings from disk...")
        data_tsne = joblib.load(filename)
    else:
        print("Performing t-SNE and saving the results to disk...")
        # Convert data from numpy array to tensor
        data_tensor = tf.convert_to_tensor(data, dtype=tf.float32)
        # Check if GPU is available
        if tf.test.is_gpu_available():
            print("Using GPU for tSNE")
            with tf.device('/GPU:0'):
                # Perform tSNE
                tsne = TSNE(perplexity=perplexity, n_iter=n_iter, random_state=42)
                data_tsne = tsne.fit_transform(data_tensor)
        else:
            print("Using CPU for tSNE")
            # Perform tSNE on CPU
            tsne = TSNE(perplexity=perplexity, n_iter=n_iter, random_state=42)
            data_tsne = tsne.fit_transform(data)
        joblib.dump(data_tsne, filename)
    return data_tsne

# Define filenames for the t-SNE embeddings
filename_OA_subset_A = Path("t-sne_embeddings_OA_subset_A.pkl")
filename_OA_subset_B = Path("t-sne_embeddings_OA_subset_B.pkl")
filename_OEA_subset_A = Path("t-sne_embeddings_OEA_subset_A.pkl")
filename_OEA_subset_B = Path("t-sne_embeddings_OEA_subset_B.pkl")
filename_OA = Path("t-sne_embeddings_OA.pkl")
filename_OEA = Path("t-sne_embeddings_OEA.pkl")


# Perform tSNE on subsets and save the results to disk
#data_tsne_OA_subset_A = perform_and_save_tsne(OA_subset_A, filename_OA_subset_A)
#data_tsne_OA_subset_B = perform_and_save_tsne(OA_subset_B, filename_OA_subset_B)
#data_tsne_OEA_subset_A = perform_and_save_tsne(OEA_subset_A, filename_OEA_subset_A)
#data_tsne_OEA_subset_B = perform_and_save_tsne(OEA_subset_B, filename_OEA_subset_B)
data_tsne_OA = perform_and_save_tsne(OA, filename_OA)
data_tsne_OEA = perform_and_save_tsne(OEA, filename_OEA)

In [ ]:
# Perform PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)  # We are reducing to 3 dimensions for visualization
# Perform PCA on the original data
data_pca_OA = pca.fit_transform(OA)
data_pca_OEA = pca.fit_transform(OEA)

In [ ]:
# Plotting UMAP

UMAP_OA_magnitudes = np.linalg.norm(data_umap_OA, ord=2, axis=1)
UMAP_OEA_magnitudes = np.linalg.norm(data_umap_OEA, ord=2, axis=1)


plt.figure(figsize=(20,30))

plt.subplot(3,2,1)
plt.title('UMAP of latent space OA')   
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.scatter(data_umap_OA[:,0], data_umap_OA[:,1], alpha=0.9, label='OA',marker=".")

plt.subplot(3, 2, 2)  # 1 row, 2 columns, second plot
plt.hexbin(data_umap_OA[:, 0], data_umap_OA[:, 1], gridsize=50, cmap='Oranges', mincnt=1)
cbar = plt.colorbar()
cbar.set_label('Counts')
plt.title('UMAP Hexbin Visualization of the OA Dataset')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')

plt.subplot(3,2,3)
plt.title('UMAP of latent space OEA')
plt.scatter(data_umap_OEA[:,0], data_umap_OEA[:,1], alpha=0.9, label='OEA',marker=".")
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')

plt.subplot(3,2,4)
plt.hexbin(data_umap_OEA[:, 0], data_umap_OEA[:, 1], gridsize=50, cmap='Oranges', mincnt=1)
cbar = plt.colorbar()
cbar.set_label('Counts')
plt.title('UMAP Hexbin Visualization of the OEA Dataset')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')

plt.subplot(3,2,5)
plt.scatter(data_umap_OA[:,0], data_umap_OA[:,1], alpha=0.9, label='OA',marker=".")
plt.scatter(data_umap_OEA[:,0], data_umap_OEA[:,1], alpha=0.1, label='OEA',marker=".")
plt.title('UMAP of OA and OEA')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.legend()

plt.subplot(3,2,6)
plt.hist(UMAP_OA_magnitudes, bins=100, alpha=0.5, label='OA')
plt.hist(UMAP_OEA_magnitudes, bins=100, alpha=0.5, label='OEA')
plt.title('Histogram of UMAP Vector Magnitudes')
plt.xlabel('Magnitude')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
#PCA Plots 

plt.figure(figsize=(20, 20))

plt.subplot(2,2,1)
plt.scatter(data_pca_OA[:,0], data_pca_OA[:,1], alpha=0.9, label='OA',marker=".")
plt.scatter(data_pca_OEA[:,0], data_pca_OEA[:,1], alpha=0.3, label='OEA',marker=".")
plt.legend()
plt.title('PCA of 1st and 2nd Principal Component')
plt.xlabel('Principal Component 0')
plt.ylabel('Principal Component 1')


plt.subplot(2,2,2)
plt.scatter(data_pca_OA[:,0], data_pca_OA[:,2], alpha=0.9, label='OA',marker=".")
plt.scatter(data_pca_OEA[:,0], data_pca_OEA[:,2], alpha=0.3, label='OEA',marker=".")
plt.legend()
plt.title('PCA of 1st and 3rd Principal Component')
plt.xlabel('Principal Component 0')
plt.ylabel('Principal Component 2')

plt.subplot(2,2,3)
plt.scatter(data_pca_OA[:,1], data_pca_OA[:,2], alpha=0.9, label='OA',marker=".")
plt.scatter(data_pca_OEA[:,1], data_pca_OEA[:,2], alpha=0.3, label='OEA',marker=".")
plt.legend()
plt.title('PCA of 2nd and 3rd Principal Component')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

plt.tight_layout()
plt.show()

In [ ]:
# MultiPLOT tSNE 

plt.figure(figsize=(20, 30))  # Create a figure with size to accommodate two plots

# First subplot: Scatter plot for OA 
plt.subplot(3, 2, 1)  # 1 row, 2 columns, first plot
plt.scatter(data_tsne_OA[:, 0], data_tsne_OA[:, 1], alpha=0.3, label='OA', marker=".")
plt.legend()
plt.title('t-SNE Visualization of OA')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')

# Second subplot: Hexbin plot for the same data (you can add OEA if needed)
plt.subplot(3, 2, 2)  # 1 row, 2 columns, second plot
plt.hexbin(data_tsne_OA[:, 0], data_tsne_OA[:, 1], gridsize=40, cmap='Oranges', mincnt=1)
cbar = plt.colorbar()
cbar.set_label('Counts')
plt.title('tSNE Hexbin Visualization of the OA Dataset')
plt.xlabel('tSNE Component 1')
plt.ylabel('tSNE Component 2')

plt.subplot(3, 2, 3)  # 1 row, 2 columns, first plot
plt.scatter(data_tsne_OEA[:, 0], data_tsne_OEA[:, 1], alpha=0.3, label='OA', marker=".")
plt.legend()
plt.title('t-SNE Visualization of OEA')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')

plt.subplot(3, 2, 4)  # 1 row, 2 columns, second plot
plt.hexbin(data_tsne_OEA[:, 0], data_tsne_OEA[:, 1], gridsize=40, cmap='Oranges', mincnt=1)
cbar = plt.colorbar()
cbar.set_label('Counts')
plt.title('tSNE Hexbin Visualization of the OEA Dataset')
plt.xlabel('tSNE Component 1')
plt.ylabel('tSNE Component 2')

# Comparison Sublplot
plt.subplot(3, 2, 5)
plt.scatter(data_tsne_OA[:, 0], data_tsne_OA[:, 1], alpha=0.3, label='OA', marker=".")
plt.scatter(data_tsne_OEA[:, 0], data_tsne_OEA[:, 1], alpha=0.3, label='OEA', marker=".")
plt.legend()
plt.title('t-SNE Visualization of OA and OEA')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')

# Adjust layout for better spacing
plt.tight_layout()

# Show the figure
plt.show()